In [1]:
import os

In [2]:
import subprocess as sp
import json

def probe(vid_file_path):
    ''' Give a json from ffprobe command line

    @vid_file_path : The absolute (full) path of the video file, string.
    '''

    command = ["ffprobe", "-loglevel",  "quiet", "-print_format", "json",
             "-show_format", "-show_streams",vid_file_path]

    pipe = sp.Popen(command, stdout=sp.PIPE, stderr=sp.STDOUT)
    out, err = pipe.communicate()
    return json.loads(out)


def duration(vid_file_path):
    ''' Video's duration in seconds, return a float number
    '''
    _json = probe(vid_file_path)

    if 'format' in _json:
        if 'duration' in _json['format']:
            return float(_json['format']['duration'])

    if 'streams' in _json:
        # commonly stream 0 is the video
        for s in _json['streams']:
            if 'duration' in s:
                return float(s['duration'])

    # if everything didn't happen,
    # we got here because no single 'return' in the above happen.
#     raise Exception('I found no duration')
    return 'I found no duration'



In [7]:

# os.chdir('..')
import split_video_gpx; reload(split_video_gpx)
from split_video_gpx import *

In [4]:
# gpx_files = glob.glob("GPX/*.gpx")
gpx_files = ['GPX/Track_2015-10-15 060649.gpx','GPX/Track_2017-02-21 125536.gpx']

In [9]:

os.chdir('Sample Data/')
print os.getcwd()
gpx_video_match = []
for gpx_f in gpx_files:
    video_name, lon_lats, timestamps = parse_gpx(gpx_f)

    # record the (gpx, video) pair when the video for a gpx can't be found
    if not os.path.isfile(video_name):
        gpx_video_match.append((gpx_f, video_name, False))
#         print "can't find video=%s for gpx=%s" % (video_name, gpx_f)
        continue
    else:
        print video_name, gpx_f, os.path.isfile(video_name), duration(video_name)
        

os.chdir('..')

/mnt/d/★★学习工作/Life in Maryland/Research Project：CyclingSafe/Cycling_safety/video_gpx/Sample Data


ExpatError: not well-formed (invalid token): line 1, column 0

In [23]:
gpx_files

[]